In [1]:
# source of data: https://www.mariowiki.com/Mario_Kart_8_Deluxe_in-game_statistics

In [2]:
from bs4 import BeautifulSoup
import requests

import pandas as pd

In [3]:
url = "https://www.mariowiki.com/Mario_Kart_8_Deluxe_in-game_statistics"
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html')

### Create variables for each HTML table

In [4]:
# character table
char_table = soup.find_all('table')[0]

# kart table
kart_table = soup.find_all('table')[1]

# tire table
tire_table = soup.find_all('table')[2]

# glider table
glider_table = soup.find_all('table')[3]

### Separate column name data & input into DataFrame

In [5]:
# find column names in table header.
# find_all returns a lot, so look through HTML and use indexing to only return column names which are stored as <th> tags
# separate drivers, karts, etc. and stats columns. Drivers are stored separately and will be hard to insert data later.

char_name_col = char_table.find_all('th')[1:2]              # used indexing to select only tag that contains 'Driver' heading
char_stats_cols = char_table.find_all('th')[2:16]           # used indexing to select tags that contain stats headings

kart_name_col = kart_table.find_all('th')[1:2]
kart_stats_cols = kart_table.find_all('th')[2:16]

tire_name_col = tire_table.find_all('th')[1:2]
tire_stats_cols = tire_table.find_all('th')[2:16]

glider_name_col = glider_table.find_all('th')[1:2]
glider_stats_cols = glider_table.find_all('th')[2:16]

char_stats_cols

[<th>WG</th>,
 <th>AC</th>,
 <th>ON</th>,
 <th>OF</th>,
 <th>MT</th>,
 <th>SL</th>,
 <th>SW</th>,
 <th>SA</th>,
 <th>SG</th>,
 <th>TL</th>,
 <th>TW</th>,
 <th>TA</th>,
 <th>TG</th>,
 <th>IV
 </th>]

In [6]:
# use list comprehension to return only text from tags

char_name_col_heading = [column.text.strip() for column in char_name_col]              # doesn't need .strip(), but keep just in case
char_stats_cols_names = [column.text.strip() for column in char_stats_cols]            # use .strip() to remove '/n' at end of list

kart_name_col_heading = [column.text.strip() for column in kart_name_col]
kart_stats_cols_names = [column.text.strip() for column in kart_stats_cols]

tire_name_col_heading = [column.text.strip() for column in tire_name_col]
tire_stats_cols_names = [column.text.strip() for column in tire_stats_cols]

glider_name_col_heading = [column.text.strip() for column in glider_name_col]
glider_stats_cols_names = [column.text.strip() for column in glider_stats_cols]

char_stats_cols_names

['WG',
 'AC',
 'ON',
 'OF',
 'MT',
 'SL',
 'SW',
 'SA',
 'SG',
 'TL',
 'TW',
 'TA',
 'TG',
 'IV']

In [61]:
# put headers into Pandas df

# first for stats tables
char_stats_df = pd.DataFrame(columns=char_stats_cols_names)
kart_stats_df = pd.DataFrame(columns=kart_stats_cols_names)
tire_stats_df = pd.DataFrame(columns=tire_stats_cols_names)
glider_stats_df = pd.DataFrame(columns=glider_stats_cols_names)

# then for Drivers, Karts, etc.
char_names_df = pd.DataFrame(columns=char_name_col_heading)
kart_names_df = pd.DataFrame(columns=kart_name_col_heading)
tire_names_df = pd.DataFrame(columns=tire_name_col_heading)
glider_names_df = pd.DataFrame(columns=glider_name_col_heading)

# rename stats columns to full stat names
full_stat_names_dict = {'WG':'Weight', 'AC':'Acceleration', 'ON':'Traction (on-road)', 'OF':'Traction (off-road)', 'MT':'Mini-Turbo',
				   'SL':'Speed (Ground)', 'SW':'Speed (Water)', 'SA':'Speed (Anti-Gravity)', 'SG':'Speed (Air)', 'TL':'Handling (Ground)', 
                   'TW':'Handling (Water)', 'TA':'Handling (Anti-Gravity)', 'TG':'Handling (Air)', 'IV':'Invincibility'}

char_stats_df.rename(columns=full_stat_names_dict, inplace=True)
kart_stats_df.rename(columns=full_stat_names_dict, inplace=True)
tire_stats_df.rename(columns=full_stat_names_dict, inplace=True)
glider_stats_df.rename(columns=full_stat_names_dict, inplace=True)

### Find Row Names because they are kept separately from table Data Values & input into DataFrames

In [62]:
# need to find row names because they aren't included in data rows.
# name info is surrounded by a '<a>' tag, and names are after title between ><
    # example: title="Princess Peach">Peach<
def row_names(table_name):
    row_names = table_name.find_all('a')

    row_names_list = []
    for row in row_names:
        row_names_list.append(row.text)
    return row_names_list

char_names = row_names(char_table)
kart_names = row_names(kart_table)
tire_names = row_names(tire_table)
glider_names = row_names(glider_table)

# the lists end up including blank strings and footnote numbers, so remove with a new program.
def remove_value(list_input, value_to_remove):
    list_input = [x for x in list_input if x not in value_to_remove]            # use 'not in' to be able to input a list
    return list_input

values_to_remove = ['', '1', '2', '3']

char_names_modified = remove_value(char_names, values_to_remove)
kart_names_modified = remove_value(kart_names, values_to_remove)
tire_names_modified = remove_value(tire_names, values_to_remove)
glider_names_modified = remove_value(glider_names, values_to_remove)

char_names_modified


# original attempt at bottom (#1)

['Mario',
 'Luigi',
 'Peach',
 'Daisy',
 'Yoshi',
 'Toad',
 'Toadette',
 'Koopa Troopa',
 'Bowser',
 'Donkey Kong',
 'Wario',
 'Waluigi',
 'Rosalina',
 'Metal Mario',
 'Pink Gold Peach',
 'Lakitu',
 'Shy Guy',
 'Baby Mario',
 'Baby Luigi',
 'Baby Peach',
 'Baby Daisy',
 'Baby Rosalina',
 'Larry',
 'Lemmy',
 'Wendy',
 'Ludwig',
 'Iggy',
 'Roy',
 'Morton',
 'Mii (medium)',
 'Tanooki Mario',
 'Link',
 'Villager (male)',
 'Isabelle',
 'Cat Peach',
 'Dry Bowser',
 'Villager (female)',
 'Gold Mario',
 'Dry Bones',
 'Bowser Jr.',
 'King Boo',
 'Inkling Girl',
 'Inkling Boy',
 'Link (Breath of the Wild)',
 'Birdo',
 'Kamek',
 'Petey Piranha',
 'Wiggler',
 'Diddy Kong',
 'Funky Kong',
 'Peachette',
 'Pauline']

In [63]:
# insert row names into corresponding dataframes

char_names_df['Driver'] = char_names_modified
kart_names_df['Body'] = kart_names_modified
tire_names_df['Tire'] = tire_names_modified
glider_names_df['Glider'] = glider_names_modified

char_names_df

,Driver
0,Mario
1,Luigi
2,Peach
3,Daisy
4,Yoshi
5,Toad
6,Toadette
7,Koopa Troopa
8,Bowser
9,Donkey Kong


### Retrieve row data and input into DataFrames

In [100]:
# This program simplifies the original attempt (#2). Cannot input into dataframe because row mismatch (blank rows at beginning of lists), so add if statement
    # to only add data lists that are > 0

def retrieve_row_data_input_to_dataframe(table_name, dataframe_output):
    table_data = table_name.find_all('tr')
    output = dataframe_output.copy()          # prevents duplicate data if run more than once

    for row in table_data:
        row_data = row.find_all('td')
        row_data_as_list = [data.text.strip() for data in row_data]       
    
        if len(row_data_as_list) > 0:          # this stops empty lists from being included which would cause a row mismatch error
            length = len(output)
            output.loc[length] = row_data_as_list
    return output

char_stats_filled = retrieve_row_data_input_to_dataframe(char_table, char_stats_df)
kart_stats_filled = retrieve_row_data_input_to_dataframe(kart_table, kart_stats_df)
tire_stats_filled = retrieve_row_data_input_to_dataframe(tire_table, tire_stats_df)
glider_stats_filled = retrieve_row_data_input_to_dataframe(glider_table, glider_stats_df)

# Stats are stored as text. Convert dataframes to integers
char_stats_filled = char_stats_filled.astype(int)
kart_stats_filled = kart_stats_filled.astype(int)
tire_stats_filled = tire_stats_filled.astype(int)
glider_stats_filled = glider_stats_filled.astype(int)

### Concat names and stats dataframes

In [101]:
characters = pd.concat([char_names_df, char_stats_filled], axis=1)
karts = pd.concat([kart_names_df, kart_stats_filled], axis=1)
tires = pd.concat([tire_names_df, tire_stats_filled], axis=1)
gliders = pd.concat([glider_names_df, glider_stats_filled], axis=1)

### Then add Mii (small) and Mii (large) to Characters df

In [102]:
# create new df for small & large miis that copies Baby Mario & Dry Bowser
mii_small_large_stats = characters.loc[characters['Driver'].isin(['Baby Mario', 'Dry Bowser'])]

# locate what row to update based on column value (using .loc) and specify the column the value will go in ('Driver')
mii_small_large_stats.loc[mii_small_large_stats['Driver'] == 'Baby Mario', 'Driver'] = 'Mii (small)'
mii_small_large_stats.loc[mii_small_large_stats['Driver'] == 'Dry Bowser', 'Driver'] = 'Mii (large)'

# concat characters and mii dataframes
characters = pd.concat([characters, mii_small_large_stats], ignore_index=True)

### Create new MT + Speed column

In [111]:
def mt_plus_speed(df):
    df['MT + Speed'] = df['Mini-Turbo'] + df['Speed (Ground)']
    
    # pop 'MT + Speed' and insert right before 'Mini-Turbo'
    mt_speed_col = df.pop('MT + Speed')
    df.insert(characters.columns.get_loc('Mini-Turbo'), 'MT + Speed', mt_speed_col)            # .columns.get_loc('col') gets the index of the column you want to put the new column in front of

    return df

characters = mt_plus_speed(characters)
karts = mt_plus_speed(karts)
tires = mt_plus_speed(tires)
gliders = mt_plus_speed(gliders)
gliders

,Glider,Weight,Acceleration,Traction (on-road),Traction (off-road),Mini-Turbo,MT + Speed,Speed (Ground),Speed (Water),Speed (Anti-Gravity),Speed (Air),Handling (Ground),Handling (Water),Handling (Anti-Gravity),Handling (Air),Invincibility
0,Super Glider,1,1,1,1,1,2,1,1,0,2,1,0,1,1,1
1,Cloud Glider,0,2,1,1,2,2,0,1,1,1,1,0,1,2,0
2,Wario Wing,2,1,2,0,1,2,1,0,1,2,1,1,0,1,1
3,Waddle Wing,1,1,1,1,1,2,1,1,0,2,1,0,1,1,1
4,Peach Parasol,1,2,2,0,2,2,0,0,1,1,1,1,0,2,0
5,Parachute,0,2,1,1,2,2,0,1,1,1,1,0,1,2,0
6,Parafoil,1,2,2,0,2,2,0,0,1,1,1,1,0,2,0
7,Flower Glider,0,2,1,1,2,2,0,1,1,1,1,0,1,2,0
8,Bowser Kite,1,2,2,0,2,2,0,0,1,1,1,1,0,2,0
9,Plane Glider,2,1,2,0,1,2,1,0,1,2,1,1,0,1,1


### Group parts that have all the same stats

In [116]:
# use groupby and lambda function to combine 'Driver' column if all stats are the same.
def group_duplicates(df, column_to_group):
    full_stat_names_list = characters[1:].columns.to_list()[1:]          # create a list of full stat names (not just abbrev.). 
                                                                         # Use [1:] to exclude 'Driver'.

    df_grouped = df.groupby(full_stat_names_list)[column_to_group].apply(lambda x: '/'.join(x)).reset_index()       # need .reset_index or else does weird thing to result

    # groupby moves the grouped column to the end. Pop the grouped column and insert it back to the front.
    grouped_column = df_grouped.pop(column_to_group)
    df_grouped.insert(0, column_to_group, grouped_column)

    return df_grouped

characters_grouped = group_duplicates(characters, 'Driver')
karts_grouped = group_duplicates(karts, 'Body')
tires_grouped = group_duplicates(tires, 'Tire')
gliders_grouped = group_duplicates(gliders, 'Glider')

gliders_grouped

,Glider,Weight,Acceleration,Traction (on-road),Traction (off-road),MT + Speed,Mini-Turbo,Speed (Ground),Speed (Water),Speed (Anti-Gravity),Speed (Air),Handling (Ground),Handling (Water),Handling (Anti-Gravity),Handling (Air),Invincibility
0,Cloud Glider/Parachute/Flower Glider/Paper Glider,0,2,1,1,2,2,0,1,1,1,1,0,1,2,0
1,Super Glider/Waddle Wing/Hylian Kite,1,1,1,1,2,1,1,1,0,2,1,0,1,1,1
2,Peach Parasol/Parafoil/Bowser Kite/MKTV Parafoil,1,2,2,0,2,2,0,0,1,1,1,1,0,2,0
3,Wario Wing/Plane Glider/Gold Glider/Paraglider,2,1,2,0,2,1,1,0,1,2,1,1,0,1,1


### Export to be used in Excel

In [117]:
# create a pd.ExcelWriter object to output multiple dataframes into one workbook
with pd.ExcelWriter('MK8D Stats.xlsx') as writer:
    characters.to_excel(writer, sheet_name='Characters', index=False)
    karts.to_excel(writer, sheet_name='Karts', index=False)
    tires.to_excel(writer, sheet_name='Tires', index=False)
    gliders.to_excel(writer, sheet_name='Gliders', index=False)
    characters_grouped.to_excel(writer, sheet_name='Characters_combined', index=False)
    karts_grouped.to_excel(writer, sheet_name='Karts_combined', index=False)
    tires_grouped.to_excel(writer, sheet_name='Tires_combined', index=False)
    gliders_grouped.to_excel(writer, sheet_name='Gliders_combined', index=False)

### Below obtains the tables using just Pandas. Much easier, but BeautifulSoup is more versatile for general webpages

In [9]:
import lxml

test_df = pd.read_html('https://www.mariowiki.com/Mario_Kart_8_Deluxe_in-game_statistics')

In [17]:
test_df[0]

Drivers (DV)  \
                                               Driver   
0                                           Mario Mro   
1                                           Luigi Lig   
2                                           Peach Pch   
3                                           Daisy Dsy   
4                                         Yoshi Ysi 1   
5                                            Toad Kno   
6                                        Toadette Knc   
7                                    Koopa Troopa Nok   
8                                          Bowser Kop   
9                                     Donkey Kong Dkg   
10                                          Wario Wro   
11                                        Waluigi Wlg   
12                                       Rosalina Rst   
13                                   Metal Mario MroM   
14                               Pink Gold Peach PchG   
15                                         Lakitu Jgm   
16                                      Shy Guy Hyh 1   
17                                  Baby Mario MroB 2   
18                                    Baby Luigi LigB   
19                                    Baby Peach PchB   
20                                    Baby Daisy DsyB   
21                                 Baby Rosalina RstB   
22                                          Larry Lry   
23                                          Lemmy Lmy   
24                                          Wendy Wdy   
25                                         Ludwig Ldw   
26                                           Iggy Igy   
27                                            Roy Roy   
28                                         Morton Mtn   
29                              Mii (medium) MiiM 2 3   
30                                 Tanooki Mario MroT   
31                                           Link Lnk   
32                                Villager (male) Mrb   
33                                       Isabelle Shz   
34                                     Cat Peach PchC   
35                                  Dry Bowser KopB 3   
36                              Villager (female) Mrg   
37                                    Gold Mario MroG   
38                                      Dry Bones Krn   
39                                    Bowser Jr. KopJ   
40                                       King Boo Ktr   
41                                 Inkling Girl Spg 1   
42                                  Inkling Boy Spb 1   
43                          Link (Breath of the Wild)   
44                                            Birdo 1   
45                                              Kamek   
46                                      Petey Piranha   
47                                            Wiggler   
48                                         Diddy Kong   
49                                         Funky Kong   
50                                          Peachette   
51                                            Pauline   
52  1: Even though there are multiple versions of ...   

                                                       \
                                                   WG   
0                                                   6   
1                                                   6   
2                                                   4   
3                                                   4   
4                                                   4   
5                                                   3   
6                                                   2   
7                                                   2   
8                                                  10   
9                                                   8   
10                                                  9   
11                                                  8   
12                                                  7   
13                                                

In [ ]:
# ORIGINAL VERSION OF THE CELL ABOVE (#1)

# need to find row names because they aren't included in data rows.
# name info is surrounded by a '<a>' tag, and names are after title between ><
    # example: title="Princess Peach">Peach<

character_names = character_table.find_all('a')

# the following doesn't work perfectly. Returns lots of blank strings and footnote numbers.
# Just remove these afterwards
char_names_list = []
for row in character_names:
    char_names_list.append(row.text)

# create program that removes undesirable values
# use 'not in' to be able to input a list of values
def remove_value(list_input, value_to_remove):
    list_input = [x for x in list_input if x not in value_to_remove]
    return list_input

# remove blanks and numbers
char_names_list_modified = remove_value(char_names_list, ['', '1', '2', '3'])
char_names_list_modified


# -----------------------------------------------------------------------------------------------------------------------------------------------


# THIS ENTIRE SECTION COULD BE MADE INTO A PROGRAM

# KARTS
kart_names = kart_table.find_all('a')

kart_names_list = []
for row in kart_names:
    kart_names_list.append(row.text)

kart_names_list_modified = remove_value(kart_names_list, [''])
# kart_names_list_modified

# TIRES
tire_names = tire_table.find_all('a')

tire_names_list = []
for row in tire_names:
    tire_names_list.append(row.text)

tire_names_list_modified = remove_value(tire_names_list, [''])
# tire_names_list_modified

# GLIDERS
glider_names = glider_table.find_all('a')

glider_names_list = []
for row in glider_names:
    glider_names_list.append(row.text)

glider_names_list_modified = remove_value(glider_names_list, [''])
# glider_names_list_modified


['Mario',
 'Luigi',
 'Peach',
 'Daisy',
 'Yoshi',
 'Toad',
 'Toadette',
 'Koopa Troopa',
 'Bowser',
 'Donkey Kong',
 'Wario',
 'Waluigi',
 'Rosalina',
 'Metal Mario',
 'Pink Gold Peach',
 'Lakitu',
 'Shy Guy',
 'Baby Mario',
 'Baby Luigi',
 'Baby Peach',
 'Baby Daisy',
 'Baby Rosalina',
 'Larry',
 'Lemmy',
 'Wendy',
 'Ludwig',
 'Iggy',
 'Roy',
 'Morton',
 'Mii (medium)',
 'Tanooki Mario',
 'Link',
 'Villager (male)',
 'Isabelle',
 'Cat Peach',
 'Dry Bowser',
 'Villager (female)',
 'Gold Mario',
 'Dry Bones',
 'Bowser Jr.',
 'King Boo',
 'Inkling Girl',
 'Inkling Boy',
 'Link (Breath of the Wild)',
 'Birdo',
 'Kamek',
 'Petey Piranha',
 'Wiggler',
 'Diddy Kong',
 'Funky Kong',
 'Peachette',
 'Pauline']

In [ ]:
# ORIGINAL ATTEMPT OF ABOVE CELL (#2)
# find rows using "tr" and retrieve row data using 'td'

character_data = character_table.find_all('tr')
for row in character_data[2:-1]:    # need [2:-1] because first 2 rows and final row are blank
    char_row_data = row.find_all('td')
    char_individual_row_data = [data.text.strip() for data in char_row_data]
    #print(char_individual_row_data)

kart_data = kart_table.find_all('tr')
for row in kart_data[2:]:
    kart_row_data = row.find_all('td')
    kart_individual_row_data = [data.text.strip() for data in kart_row_data]
    #print(kart_individual_row_data)

tire_data = tire_table.find_all('tr')
for row in tire_data[2:]:
    tire_row_data = row.find_all('td')
    tire_individual_row_data = [data.text.strip() for data in tire_row_data]
    #print(tire_individual_row_data)

glider_data = glider_table.find_all('tr')
for row in glider_data[2:]:
    glider_row_data = row.find_all('td')
    glider_individual_row_data = [data.text.strip() for data in glider_row_data]
    #print(glider_individual_row_data)


# ------------------------------------------------------------------------


# this originally returned error because names were missing (character name, kart name, etc.). Fixed in above section
# also separated names into own dataframes because couldn't fill rows with data if they were present. Can concat later

character_data = character_table.find_all('tr')
char_stats_copy = characters_stats.copy()          # this prevents duplicate data if cell is run more than once

for row in character_data[2:-1]:
    char_row_data = row.find_all('td')
    char_individual_row_data = [data.text.strip() for data in char_row_data]

    length = len(char_stats_copy)
    char_stats_copy.loc[length] = char_individual_row_data

char_stats_copy


# ------------------------------------------------------------------------


kart_data = kart_table.find_all('tr')

for row in kart_data[2:]:
    kart_row_data = row.find_all('td')
    kart_individual_row_data = [data.text.strip() for data in kart_row_data]

    length = len(karts_stats)
    karts_stats.loc[length] = kart_individual_row_data

tire_data = tire_table.find_all('tr')
for row in tire_data[2:]:
    tire_row_data = row.find_all('td')
    tire_individual_row_data = [data.text.strip() for data in tire_row_data]

    length = len(tires_stats)
    tires_stats.loc[length] = tire_individual_row_data

glider_data = glider_table.find_all('tr')
for row in glider_data[2:]:
    glider_row_data = row.find_all('td')
    glider_individual_row_data = [data.text.strip() for data in glider_row_data]

    length = len(gliders_stats)
    gliders_stats.loc[length] = glider_individual_row_data